# Modeling Sequential Data Using Recurrent Neural Networks
**Mô hình hóa dữ liệu tuần tự với mạng mạng thần kinh hồi quy**

Ở chương trước, chúng ta đã cùng tìm hiểu về `CNN` từ các building block cho tới triển khai `CNN` với `tensorflow`. Ở chương này chúng ta sẽ cùng tìm hiểu về `RNN` (Recurrent Neural Network) và các biến thể của `RNN` với dữ liệu dạng tuần tự.
1. Giới thiệu về dữ liệu dạng tuần tự, các bài toán hay gặp.
2. Mô hình hồi quy và biến thể.
3. Lan Truyền ngược tuần tự (BPTT) và gradient vanishing.
4. Truncated backpropagation through time (TBPTT).
5. Triển khai `RNNs` và các biến thể bằng `numpy`.
6. Attention và mô hình `transformer` (cơ chế self-attention) [*].

## Giới thiệu về dữ liệu dạng tuần tự, các bài toán hay gặp.

### Vấn đề với dữ liệu tuần tự trong các mô hình `NNs` không ghi nhớ
Chúng ta bắt đầu với bài toán `slot filling` và các kiến thức từ chương trước. giả sử chúng ta có đề bài thực tế như sau:

Một hãng hàng cần xây dựng một hệ thống tự động ghi nhận thông tin chuyến bay của hành khách khi họ nhắn tin vào hệ thống tư vấn của họ và đưa ra những lịch trình bay hợp lý với thông tin mà người hành khách đó mong muốn. Nghiệp vụ được mô tả như hình dưới đây.
![hình 1](https://raw.githubusercontent.com/HuangRihChang/machine_learning_basic/master/RNN/images/1.png)

`slots` là các trường chúng ta cần có để biết được thông tin mà hành khách cung cấp, trong trường hợp trên chúng ta định nghĩa 2 `slots` là *time of arrival* và *destination*. Nhiệm vụ của chúng ta là điền thông tin vào các trường còn thiếu đó hay còn gọi là `slots`.

Câu hỏi đặt ra là liệu chúng ta có thể giải quyết bài toán trên bằng ANNs thông thường hay không? 
Để có thể giải quyết bài toán này bằng `ANNs`, chúng ta sẽ cho lặp từng chữ một trong câu đi vào mạng `ANNs` và dự đoán đầu xem từ đó có xác suất thuộc `slot` nào.
![hình 2](https://raw.githubusercontent.com/HuangRihChang/machine_learning_basic/master/RNN/images/2.png)
Ví dụ như việc chúng ta đưa vào chữ **Taipei** vào mô hình `ANNs`, kết quả của mô hình sẽ cho ta biết được rằng là **Taipei** có xác suất thuộc vào `slot` là `destination`. Hãy cùng xem xét lại đôi chút và nâng độ khó của bài toán lên. Bây giờ chúng ta sẽ có 3 `slots`: *destination*, *place of department* và *time of arrival*.
Khi đó chúng ta hãy cùng xét câu: **I would like to leave Taipei on November 2nd**. 
![hình 3](https://raw.githubusercontent.com/HuangRihChang/machine_learning_basic/master/RNN/images/3.png)
Vấn đề đã xuất hiện với các mô hình `ANNs`, cùng phân tích đôi chút về vấn đề mà chúng ta mới gặp. Việc `ANNs` không xác định được **Taipei** là *destination* hay là *place of department* chính là do `ANNs` không ghi nhớ các dữ liệu trước đó đã đi qua, cụ thể ở đây là phụ thuộc vào chữ **leave** hay chữ **arrive**. Và còn trường hợp khi **Taipei** không thuộc bất kỳ `slot` nào cả. Do đó chúng ta có thể kết luận, đặc điểm của dạng dữ liệu tuần tự chính là vấn đề về thứ tự của dữ liệu và để xử lý mô hình hóa chúng, <font color='darkblue'>chúng ta cần xây dựng một mô hình có khả năng ghi nhớ.</font>

### Vấn đề biểu diễn tuần tự
Chúng ta đã biết được độ các vấn đề về dữ liệu tuần tự trong `ANNs` và cũng biết được cần xây dựng một mô hình có thể giải quyết các bài toán sử dụng dữ liệu tuần tự. Thế nhưng việc chúng ta sẽ biểu diễn dữ liệu có thứ tự này trong các mô hình học máy như thế nào?

Quay lại với bài toán ở **phần 1.1**, giả sử chúng ta có $N$ câu tương ứng với $N$ điểm dữ liệu trong tập huấn luyện. Chúng ta sẽ cùng giả sử thêm một vấn đề nữa đó là độ dài các câu sẽ tương đồng với nhau là $T$ chữ. Từ đó chúng ta sẽ có $N$ vector có $T$ chiều, (Giả định chúng ta sử dụng tf-idf cho việc vector hóa 1 chữ thành 1 số thực). Khi đó ta có 1 điểm dữ liệu sẽ tương ứng
$$
\begin{aligned}
\boldsymbol{x}&=  \begin{bmatrix}
        x^0, x^1,\dots,x^T
    \end{bmatrix}
\end{aligned}
$$
trong đó các chỉ số trên đầu mỗi phần tử trong vector thể hiện vị trí của phần tử đó trong câu (hàng, cột). Để ví dụ cho việc biểu diễn dữ liệu có tính tuần tự, vector đầu ra hay nhãn cũng sẽ được biểu diễn dưới dạng một vector có $T$ phần tử. Ứng với mỗi chỉ số $t$ chúng ta sẽ có 1 cặp $x^t, y^t$ tương ứng. Với $\boldsymbol{y}$ là vector đầu ra. Để dễ hình dung hơn, ý tưởng trên sẽ được thể hiện bằng hình dưới đây.
![hình 4](https://raw.githubusercontent.com/HuangRihChang/machine_learning_basic/master/RNN/images/4.png)

### Các loại mô hình tuần tự:
Các vấn đề liên quan đến bài toán tuần tự có rất nhiều ứng dụng, có thể kể đến như: dịch máy (machine  translation), chú thích hình ảnh (image captioning), sinh văn bản (text generation). Tuy nhiên, với mỗi ứng dụng sẽ ứng với một kiến trúc mô hình khác nhau. Việc hiểu được sự khác biệt giữa các mô hình với nhau là việc rất quan trọng trong việc tiếp cận một bài toán tuần tự. *xem thêm: [The Unreasonable Effectiveness of Recurrent Neural Networks, by Andrej Karpathy](http://karpathy.github.io/2015/05/21/rnn-effectiveness/)*. Trong phần này, chúng ta sẽ cùng liệt kê một vài mô hình hay gặp nhất.
![hình 5](https://raw.githubusercontent.com/HuangRihChang/machine_learning_basic/master/RNN/images/5.png)
1. **Many-to-one**: đầu vào mô hình là một chuỗi tuần tự, nhưng đầu ra là một vector có chiều cố định. Có thể xét tới như các bài toán: phân tích sắc thái (sentiments analysis), intent detection, Forex forecasting.
2. **One-to-many**: đầu vào của mô hình là một vector có số chiều cố định, nhưng đầu ra là một chuỗi vector tuần tự. VD: image captioning.
3. **Many-to-many**: đầu vào là một chuỗi tuần tự, đầu ra cũng là một chuỗi tuần tự. VD: Machine translation, Name Entities Recognition, Part Of Speech, Words Segmentation.

## Mô hình hồi quy và biến thể.

![hình 0](https://raw.githubusercontent.com/HuangRihChang/machine_learning_basic/master/RNN/images/0.png)
Trong phần này, trước đi đào sâu vào việc cài đặt và triển khai các mô hình hồi quy `RNNs`. Chúng ta sẽ cùng thảo luận và dạo qua các kiến trúc điển hình trong dòng mô hình này. Từ đó hiểu hơn về  mô hình hồi quy `RNN` nguyên bản và tiền đề của sự ra đời của các biến thể khác trong dòng mô hình hồi quy.
### Recurrent Neural Network (RNN)
#### Cơ chế của Recurrent Neural Network (RNN)
Cơ chế cơ bản của RNN nguyên bản đó chính là cơ chế lặp. Chúng ta có thể bắt đầu với kiến trúc của một RNN cơ bản so sánh với một FFN (ANN một tầng ẩn) như sau:
![hình 6](https://raw.githubusercontent.com/HuangRihChang/machine_learning_basic/master/RNN/images/6.png)
Quy ước: cả hai đều chỉ có một tầng ẩn như nhau, nhưng tại mỗi tầng đều có thể chứa nhiều `units`.

Trong `FFN`, thông tin được đi một chiều duy nhất từ inputs vào hidden layer và tiếp tục đi đến outputs. Mặt khác, trong `RNN` thông tin đi vào hidden layer sẽ bao gồm đồng thời thông tin từ lớp inputs tại thời điểm $t$ và thông tin từ hidden layer ở thời điểm $t-1$.

Luồng thông tin trong các bước thời gian liền kề tại hidden layer cho phép `RNN` có bộ nhớ về các thông tin trong quá khứ gần. Luồng thông tin này thường được hiển thị dưới dạng một vòng lặp, còn được gọi là một **recurrent edge** trong ký hiệu đồ thị, do đó mô hình này có tên gọi là `RNN` (Recurrent Neural Network).

Giống với `MLP` hay `ANN` và `CNN`, `RNN` cũng có thể có nhiều hidden layers, hình minh họa phía dưới. Chú ý ở đây chúng ta phân biệt `RNN` đơn lớp và `MLP` đơn lớp (với `MLP` đơn lớp hay đơn tầng có nghĩa là sẽ không có lớp hidden layer, VD như Adaline hay Logistic Regression).
![hình 7](https://raw.githubusercontent.com/HuangRihChang/machine_learning_basic/master/RNN/images/7.png)
Để khảo sát kiến trúc và luồng thông tin đi lưu thông trong `RNN`, chúng ta sẽ "mở cuộn" **recurrent edge** ra như kiến trúc ở kế bên ở hình trên.

Xem xét `RNN` đơn lớp, giống như ở `FFN` (hay `ANN`), hidden layer sẽ nhận đầu vào là một thông tin duy nhất từ inputs layer. Còn trong `RNN`, tại mỗi time stamp $t$, hidden layer sẽ nhận hai đầu vào khác biệt nhau: (1) từ inputs ở time stamp hiện tại ($x^t$) và (2) thông tin từ chính hidden layer đó ở time stamp trước đó (h^{t-1}).
Ghi chú: Trong thời điểm bắt đầu, time stamp $t=0$, hidden state tại thời điểm trước do chưa có nên chúng thường được khởi tạo bằng $0$ hoặc rất nhỏ gần bằng $0$ một cách ngẫu nhiên.

Tương tự, đối với `RNN` đa lớp, dựa trên hình phía trên chúng ta có thể tóm tắt luồng đi ủa thông tin như sau:
1. Layer=1: ở tại hidden layer 1, $h^t_1$, sẽ nhận đầu vào từ inputs layer, $x^t$, và thông tin của hidden state từ cùng chính hidden layer 1 nhưng ở time-stamp trước đó, $h^{t-1}_1$.
1. Layer=2: ở tại hidden layer 2, $h^t_2$, sẽ nhận đầu vào từ output của hidden layer 1 tại cùng time-stamp, $o^t_1$, và thông tin của hidden state từ cùng chính hidden layer 2 nhưng ở time-stamp trước đó, $h^{t-1}_2$.

#### Kích hoạt units trong RNN
hiện tại chúng ta đã hiểu được kiến trúc và luồng thông tin trong `RNN`. Để hiểu rõ hơn về cách `RNN` tính toán, ghi nhớ và kích hoạt trong hidden layer. Để đơn giản, chúng ta sẽ chỉ xem xét một hidden layer duy nhất. Tuy nhiên, khái niệm tương tự cũng áp dụng cho các `RNN` đa lớp.

Tại mỗi cạnh có hướng (kết nối giữa Cell trong hình unfold) trong biểu diễn của một `RNN` mà chúng ta vừa xem xét được liên kết với một ma trận trọng số. Các trọng số đó không phụ thuộc vào thời gian, $t$. Đồng nghĩa là chúng được chia sẻ trên trục thời gian. Các ma trận trọng số khác nhau trong `RNN` đơn lớp sẽ như hình dưới
![hình 8](https://raw.githubusercontent.com/HuangRihChang/machine_learning_basic/master/RNN/images/8.png)

$$
\begin{aligned}
\boldsymbol{a}^{(t)}_h &= \boldsymbol{W}_{xh}\boldsymbol{x}^{(t)} + \boldsymbol{W}_{hh}\boldsymbol{h}^{(t-1)}+\boldsymbol{b}_h\\
\boldsymbol{h}^{(t)} &= \phi_h(\boldsymbol{a}^{(t)}_h)\\
\boldsymbol{o}^{(t)} &= \boldsymbol{W}_{ho}\boldsymbol{h}^{(t)} + \boldsymbol{b}_o\\
% \boldsymbol{o}^{(t)} &= \phi_o(\boldsymbol{W}_{ho}\boldsymbol{h}^{(t)} + \boldsymbol{b}_o)
\end{aligned}
$$

#### Backpropogation through time (BPTT)
Phương pháp huấn luyện RNNs được giới thiệu vào năm 1990, *Backpropagation Through Time: What It Does and How to Do It (Paul Werbos, Proceedings of IEEE, 78(10): 1550-1560, 1990*.
Việc tính đạo hàm của gradients trong RNNs rất phức tạp. Nhưng những luật cơ bản nhất trong giải thuật này cũng tương tự với giải thuật `backpropogation` (lan truyền ngược). Ý tưởng chính của giải thuật này là "Loss tổng bằng tổng losses" từ time-stamp $t=1$ đến $t=T$:

$$
\begin{aligned}
\boldsymbol{L} &= \sum_{t=1}^{T}L^{(t)}
\end{aligned}
$$

Vì Loss tại time-stamp $t$ phụ thuộc vào các hidden units tại tất cả các time-stamps trước đó từ $1$ đên $t$, do đó gradient tại time-stamp $t$ sẽ được tính như sau:
$$
\begin{aligned}
\frac{\partial L^{(t)}}{\partial \boldsymbol{W}_{hh}} &= \frac{\partial L^{(t)}}{\partial \boldsymbol{o}^{(t)}} \frac{\partial\boldsymbol{o}^{(t)}}{\partial\boldsymbol{h}^{(t)}}(\sum_{k=1}^{t}\frac{\partial\boldsymbol{h}^{(t)}}{\partial\boldsymbol{h}^{(k)}}\frac{\partial\boldsymbol{h}^{(k)}}{\partial\boldsymbol{W}_{hh}})
\end{aligned}
$$

Trong đó, $\frac{\partial\boldsymbol{h}^{(t)}}{\partial\boldsymbol{h}^{(k)}}$ được tính như sau:
$$
\begin{aligned}
\frac{\partial\boldsymbol{h}^{(t)}}{\partial\boldsymbol{h}^{(k)}}&=\prod_{i=k+1}^{t} \frac{\partial\boldsymbol{h}^{(i)}}{\partial\boldsymbol{h}^{(i-1)}}
\end{aligned}
$$

#### Những thách thức khi học tương tác tầm xa (long-range dependencies)

Như đã được giới thiệu ở phần trước, `BPTT` là phương pháp học được đề xuất cho `RNNs`. Việc tính toán $\frac{\partial\boldsymbol{h}^{(t)}}{\partial\boldsymbol{h}^{(k)}}$, cho `gradients` trong `Loss function`, nảy sinh vấn đề được gọi `vanishing` hay `exploding` gradient. Để hiểu hơn 2 vấn đề này chúng ta sẽ cùng chúng ta phân tích minh họa phía dưới:
![hình 9](https://raw.githubusercontent.com/HuangRihChang/machine_learning_basic/master/RNN/images/9.png)
Cơ bản, $\frac{\partial\boldsymbol{h}^{(t)}}{\partial\boldsymbol{h}^{(k)}}$ sẽ được tính bằng $t-k$ lần tích lại với nhau, $w^{t-k}$. Từ đó chúng ta có thể nhận thấy, nếu $|w|<1$, tích đó sẽ trở nên càng nhỏ tức là $\frac{\partial\boldsymbol{h}^{(t)}}{\partial\boldsymbol{h}^{(k)}}$ hay $\frac{\partial L^{(t)}}{\partial \boldsymbol{W}_{hh}}$ sẽ tiến về $0$ nếu $t-k$ càng lớn, đồng nghĩa với việc ở các time-stamp ở vị trí rất xa so với vị trí đang đứng sẽ gần như không có cập nhật được gì cả. Trong trường hợp ngược lại, với $|w|>1$, thì tích $w^{t-k}$ sẽ càng lớn khi $t-k$ càng lớn. Nhớ rằng $t-k$ đại diện cho những thông tin phụ thuộc xa (long range dependencies). Từ đó chúng ta có thể "ngây thơ" nghĩ ra giải pháp cơ bản cho 2 vấn đề trên bằng hình thứ 3 ở ảnh trên. Đó là giới hạn giá trị của $w$, $|w|=1$.

Như vậy về lý thuyết là RNN có thể mang thông tin từ các time-stamp trước đến các time-stamps sau, nhưng thực tế là thông tin chỉ mang được qua một số lượng time-stamp nhất định, sau đó thì sẽ bị vanishing gradient, hay nói cách khác là mô hình chỉ học được từ các time-stamp gần nó => **short term memory**.

Trong thực tế, để giải quyết vấn đề trên có những phương pháp giải quyết chính:
- Gradient clipping: phương pháp này đơn giản là chúng ta sẽ giới hạn lại ngưỡng của gradients.
- Truncated BPTT (TBPTT): TBPTT đơn giản giới hạn lại ngưỡng của $t-k$ số lần mà thông tin lan truyền ngược lại trong mỗi lần chuyển tiếp (forward pass) giữa các time-stamps. Ví dụ, chúng ta có 1 chuỗi dữ liệu tuần hoàn 100 time-stamps, nhưng trong mỗi lần chúng ta backpropagate 20 time-stamps gần nhất. 

#### Long-short term memories

Mặc dù, Gradient clipping và TBPTT đều có thể giải quyết rất tốt 2 vấn đề về vanishing gradients và exploding gradients. Thế nhưng việc giới hạn giá trị và cắt giảm có thể sẽ ảnh hưởng đến vấn đề cập nhập trong số một cách chính xác và hiểu quả. Vào năm 1997, Sepp Hochreiter và Jürgen Schmidhuber giới thiệu `LSTM`, đây được xem là mô hình có thể giải quyết vấn đề vanishing gradients và exploding gradients trong vấn đề mô hình hóa các thông tin phụ thuộc xa.

Trước tiên có thể nói đến đó là khối kiến trúc (building block) của `LSTM` là một ô nhớ, `memory cell`, mỗi một `memory cell` về cơ bản có thể xem như đại diện hoặc thay thế cho một lớp ẩn của `standard RNN`.

Cũng như bên `RNN`, ở `LSTM`, trong mỗi `memory cell`, đều tồn tại một cạnh lặp (cạnh hồi quy) `recurrent edge` đáp ứng được yêu cầu như chúng ta đã phân tích ở trên để tránh được 2 vấn đề vanishing và exploding gradients. Giá trị được định nghĩa trên cạnh đó trong `LSTM` được gọi là giá trị trạng thái ô, `cell state`, $\boldsymbol{C}^{(t)}$.

![hình 10](https://raw.githubusercontent.com/HuangRihChang/machine_learning_basic/master/RNN/images/10.png)

